In [8]:
import requests
import json
import os
import pandas as pd

In [9]:
RAW_DATA_PATH = os.path.join("data", "raw")
PROCESSED_DATA_PATH = os.path.join("data", "processed")

Load Data CSV Taxon and Acession

In [10]:
data_accession_taxon = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "data_accession_taxon.csv"))

In [12]:
data_accession_taxon.sample(5)

,accession,tax_id,organism_name
987,GCF_030316605.1,1276219,Spiroplasma ixodetis Y32
690,GCF_000242915.1,929558,Sulfurimonas gotlandica GD1
277,GCF_027554905.1,145262,Methanothermobacter thermautotrophicus
403,GCF_008931805.1,1202962,Moritella marina ATCC 15381
732,GCF_002278035.1,271865,[Ochrobactrum] quorumnocens


In [19]:
data_accession_taxon[data_accession_taxon.organism_name.str.contains("ixo")]

,accession,tax_id,organism_name
987,GCF_030316605.1,1276219,Spiroplasma ixodetis Y32


Url Download FASTA

In [5]:
urls_summary = "https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/accession/{accession}/download_summary?include_annotation_type[0]=GENOME_FASTA"

In [20]:
urls_summary.format(accession="GCF_030316605.1")

'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/accession/GCF_030316605.1/download_summary?include_annotation_type[0]=GENOME_FASTA'

In [21]:
summary_response = requests.get(urls_summary.format(accession="GCF_030316605.1"))

In [24]:
print(json.dumps(summary_response.json(), indent = 4))

{
    "record_count": 1,
    "resource_updated_on": "2023-10-14T03:40:16Z",
    "hydrated": {
        "estimated_file_size_mb": 1,
        "url": "https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/accession/GCF_030316605.1/download?include_annotation_type=DEFAULT,GENOME_FASTA",
        "cli_download_command_line": "datasets download genome accession GCF_030316605.1 --include genome"
    },
    "dehydrated": {
        "estimated_file_size_mb": 1,
        "url": "https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/accession/GCF_030316605.1/download?hydrated=DATA_REPORT_ONLY&include_annotation_type=DEFAULT,GENOME_FASTA",
        "cli_download_command_line": "datasets download genome accession GCF_030316605.1 --include genome --dehydrated",
        "cli_rehydrate_command_line": "datasets rehydrate --help"
    },
    "available_files": {
        "all_genomic_fasta": {
            "file_count": 1,
            "size_mb": 0.5104723
        },
        "genome_gff": {
            "file_count

In [26]:
urls_download = summary_response.json()["hydrated"]["url"]

In [28]:
urls_download

'https://api.ncbi.nlm.nih.gov/datasets/v2alpha/genome/accession/GCF_030316605.1/download?include_annotation_type=DEFAULT,GENOME_FASTA'